# Mapreader Workshop - 30th April - 1st May 2024

----

First check you have the correct version of MapReader. 
For the April 2024 workshop, this is v1.2.0

This can be downloaded from pypi using `pip install mapreader==1.2.0` or by checking out the repo at [this commit](https://github.com/Living-with-machines/MapReader/releases/tag/v1.2.0)

In [ ]:
import mapreader
assert mapreader.__version__ == '1.2.0'

-------------

# Classify

We can also use our ``ClassifierContainer`` to infer labels on new datasets with a pre-trained/fine-tuned model.

We are going to look at how to use our railspace model (https://huggingface.co/Livingwithmachines/mr_resnest101e_finetuned_OS_6inch_2nd_ed_railspace) to predict labels on patches.
To do this, we need to load the model from huggingface and then pass it in as the ``model`` argument in our ``ClassifierContainer``.

In [ ]:
try:
    import timm
except ImportError: # timm isn't installed by default, so you might need to install it
    !pip install timm
    import timm

In [ ]:
my_model = timm.create_model("hf_hub:Livingwithmachines/mr_resnest101e_finetuned_OS_6inch_2nd_ed_railspace", pretrained=True)

Now we set up the ``ClassifierContainer``

In [ ]:
from mapreader import ClassifierContainer

In [ ]:
my_classifier = ClassifierContainer(
    model=my_model,
    labels_map={0:"no", 1:"railspace"}, # manually set this, you can find it in the model card - https://huggingface.co/Livingwithmachines/mr_resnest101e_finetuned_OS_6inch_2nd_ed_railspace
    dataloaders=None,
)

The model can now be used to infer, or predict, the labels of "unseen" patches.

In [ ]:
import pandas as pd

patch_df = pd.read_excel("./patch_df.csv", index_col=0)  # load our patch_df.csv file
patch_df.head()

> __**NOTE**__: MapReader can be used to predict the labels on entire datasets and so creating a ``subset_patch_df`` is not needed in most use cases.

### Create a dataset (``infer_all``) from our ``patch_df``

In [ ]:
from mapreader import PatchDataset

In [ ]:
infer_all = PatchDataset(patch_df, transform="val", patch_paths_col="image_path")

### Load dataset into ``my_classifier``

In [ ]:
my_classifier.load_dataset(infer_all, "infer_all")

### Run model inference

__**TODO**__: Run inference on your ``"infer_all"`` dataset

In [ ]:
# my_classifier.inference()

Save results!

In [ ]:
my_classifier.save_predictions("infer_all")

### Save results to metadata

To add the predictions back into a ``MapImages`` object, we simply need to load our predictions csv file as metadata.

Since we have started a new notebook, we can create a new ``MapImages`` object by loading our patches.

In [ ]:
from mapreader import load_patches

In [ ]:
my_maps = load_patches(
    "./patches_100_meters/png", parent_paths="./maps/*png"
)

In [ ]:
my_maps.add_metadata("./infer_all_predictions_patch_df.csv", ignore_mismatch=True, tree_level="patch")

In [ ]:
my_maps.add_shape()

We can use the ``.show_parent()`` method to see how our predictions look on our parent map sheet (``map_74488700.png``).

In [ ]:
my_maps.show_parent(
    "map_75650661.png",
    column_to_plot="pred",
    vmin=0,
    vmax=1,
    alpha=0.5,
    patch_border=False,
)

And the ``.convert_images()`` method to save our results.

In [ ]:
parent_df, patch_df = my_maps.convert_images(save=True, save_format="csv")

We can also save our outputs as a ``geojson`` file using the ``.save_patches_to_geojson()`` method. We'll call the file ``"railspace_patches.geojson"``.

> _**NOTE**_: This will require you to convert your patch coordinates into a polygon format. If these aren't already available, they can be added using the ``.add_patch_polygons()`` method.

In [ ]:
my_maps.add_patch_polygons()
my_maps.save_patches_to_geojson("railspace_patches.geojson")

Beyond MapReader, these outputs can be used to generate interesting visualizations in other tools.

For example, here are two visualizations of the rail space data from [our paper]:

- https://felt.com/map/MapReader-Launch-Event-map-Urban-Areas-and-Rail-space-9AqftKrvPTlWfwOGkdkCGkD
- https://maps.nls.uk/projects/mapreader/index.html#zoom=6.0&lat=56.00000&lon=-4.00000

# Documentation

Please refer to the [MapReader documentation](https://mapreader.readthedocs.io/en/latest/) for more information.